## Question 1

### Importing relevant libraries

In [1]:
import io
from io import BytesIO
import requests
import pandas as pd
import numpy as np
from datetime import timedelta, date
import datetime as datetime

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

C:\Users\MANAS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\MANAS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\MANAS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Getting the list of NSE500 symbols

In [2]:
def get_NSE_symbols(url):
    
    symbollist = []
    while True:
        try:
            r = requests.get(url)
        except requests.ConnectionError:
            print('No connection, retrying')
        break
    print("--Connection Established Successfully--") 
    
    with open('N500.csv', "wb") as outfile:
        outfile.write(BytesIO(r.content).getbuffer())
        
    print("--Extracted NSE500 Symbols Successfully--") 
    data = pd.read_csv('N500.csv')
    for i in data.Symbol:
        symbollist.append(i)
    f = open("N500.csv",'w')
    f.close()
    return symbollist

### Extract DAT data 

In [3]:
def get_DAT(dates, symbol):
    
    column = ['Record Type', 'Sr.No', 'Symbol','Type', 'QT', 'DQ', '% DQ/TQ']    
    c = 0
    for start in dates:        
        if start.month <= 9:
            if start.day <= 9:
                index = "0"+str(start.day)+"0"+str(start.month)+str(start.year)
            else:
                index = str(start.day)+"0"+str(start.month)+str(start.year)
        else:
            index = str(start.day)+str(start.month)+str(start.year)

        endpoint = 'https://www1.nseindia.com/archives/equities/mto/MTO_' + str(index) +'.DAT'

        try:
            r = requests.get(endpoint)
        except requests.status_codes == 404:
            print(f'{start} is not a trading day')
            continue

        
        with open('temp.csv', "wb") as outfile:
            outfile.write(BytesIO(r.content).getbuffer())    
        
        temp = pd.read_csv('temp.csv', skiprows = 4, names = column)
        temp = temp[temp.Type == 'EQ']
        temp = temp[temp.Symbol.isin(symbol)]
        
        if c == 0:
            col = []
            for sym in symbol:
                col.append(sym)
                col.append(f'{sym}_DQ')
            df = pd.DataFrame(columns = col) 
            c = 1
        
        k = 0

        for i in range(temp.shape[0]):
            df = df.append({temp.Symbol.iloc[i] : start, f'{temp.Symbol.iloc[i]}_DQ' : temp.DQ.iloc[i]}, ignore_index = True)   
            k+=1
            
        print(f'--Extracted data for {k} symbols on date : {index}--')
        df = df.apply(lambda x: pd.Series(x.dropna().values))
        f = open("temp.csv",'w')
        f.close()
    return df

### Arguments

In [4]:
url = 'https://www1.nseindia.com/content/indices/ind_nifty500list.csv'
symbol = get_NSE_symbols(url)

start = date(2005, 5, 1)
end = date(2005, 7, 8)

dateList = pd.bdate_range(start=start, end=end).to_pydatetime()
df = get_DAT(dateList,symbol)

--Connection Established Successfully--
--Extracted NSE500 Symbols Successfully--
--Extracted data for 158 symbols on date : 02052005--
--Extracted data for 158 symbols on date : 03052005--
--Extracted data for 158 symbols on date : 04052005--
--Extracted data for 158 symbols on date : 05052005--
--Extracted data for 159 symbols on date : 06052005--
--Extracted data for 159 symbols on date : 09052005--
--Extracted data for 159 symbols on date : 10052005--
--Extracted data for 159 symbols on date : 11052005--
--Extracted data for 159 symbols on date : 12052005--
--Extracted data for 159 symbols on date : 13052005--
--Extracted data for 159 symbols on date : 16052005--
--Extracted data for 159 symbols on date : 17052005--
--Extracted data for 159 symbols on date : 18052005--
--Extracted data for 159 symbols on date : 19052005--
--Extracted data for 159 symbols on date : 20052005--
--Extracted data for 159 symbols on date : 23052005--
--Extracted data for 159 symbols on date : 24052005--


### Output

In [7]:
df

,3MINDIA,3MINDIA_DQ,ABB,ABB_DQ,ACC,ACC_DQ,AIAENG,AIAENG_DQ,APLAPOLLO,APLAPOLLO_DQ,...,ZENSARTECH,ZENSARTECH_DQ,ZOMATO,ZOMATO_DQ,ZYDUSLIFE,ZYDUSLIFE_DQ,ZYDUSWELL,ZYDUSWELL_DQ,ECLERX,ECLERX_DQ
0,2005-05-02,3595,2005-05-02,8811,2005-05-02,302597,NaN,NaN,NaN,NaN,...,2005-05-02,22926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005-05-03,4455,2005-05-03,7725,2005-05-03,205422,NaN,NaN,NaN,NaN,...,2005-05-03,18888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005-05-04,1892,2005-05-04,13908,2005-05-04,157152,NaN,NaN,NaN,NaN,...,2005-05-04,14123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005-05-05,586,2005-05-05,3709,2005-05-05,111081,NaN,NaN,NaN,NaN,...,2005-05-05,118763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005-05-06,849,2005-05-06,3127,2005-05-06,349005,NaN,NaN,NaN,NaN,...,2005-05-06,43158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2005-05-09,371,2005-05-09,9986,2005-05-09,270217,NaN,NaN,NaN,NaN,...,2005-05-09,43557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2005-05-10,320,2005-05-10,8545,2005-05-10,362121,NaN,NaN,NaN,NaN,...,2005-05-10,37724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2005-05-11,1042,2005-05-11,5296,2005-05-11,97370,NaN,NaN,NaN,NaN,...,2005-05-11,22007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2005-05-12,1264,2005-05-12,14365,2005-05-12,259979,NaN,NaN,NaN,NaN,...,2005-05-12,21644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2005-05-13,2993,2005-05-13,13755,2005-05-13,181831,NaN,NaN,NaN,NaN,...,2005-05-13,25177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.to_excel("Solution 1.xlsx")